In [ ]:
# lane_detector.py
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms

from model.model_culane import parsingNet
from configs.culane_res34 import cfg
from data.constant import culane_row_anchor as row_anchor

class LaneDetector:
    def __init__(self, weight_path, use_gpu=True):
        self.device = torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu')

        # Load model
        self.model = parsingNet(pretrained=False, backbone='resnet34', use_aux=False).to(self.device)
        self.model.load_state_dict(torch.load(weight_path, map_location=self.device)['model'])
        self.model.eval()

        # Image input size
        self.img_trans = transforms.Compose([
            transforms.Resize((288, 800)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
        self.row_anchor = row_anchor
        self.num_griding = cfg.num_griding
        self.num_lanes = cfg.num_lanes

    def detect(self, frame):
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_transformed = self.img_trans(img).unsqueeze(0).to(self.device)

        with torch.no_grad():
            out = self.model(img_transformed)[0]
        out = out[0].cpu().numpy()

        col_sample = np.linspace(0, 800 - 1, self.num_griding)
        col_sample_w = col_sample[1] - col_sample[0]

        lanes = []
        for i in range(out.shape[0]):
            if np.max(out[i]) < 0.1:  # skip if no confidence
                continue
            out_i = out[i]
            lane = []
            for r in range(out_i.shape[0]):
                idx = int(np.argmax(out_i[r]))
                if idx > 0:
                    x = int(col_sample[idx])
                    y = int(self.row_anchor[r] * frame.shape[0] / 288)
                    lane.append((x, y))
            if len(lane) > 2:
                lanes.append(lane)
        return lanes
